In [4]:
import os
import pandas as pd
from pathlib import Path
import openpyxl
import tabula

In [5]:
download_path = "./Download"
export_path = "./Download_CSV"
merged_csv_path = f"{export_path}/merged_dataset.csv"
merged_sums_path = f"{export_path}/merged_sums.csv"

file_path = (f"{download_path}/2017 Q1.xlsx")
df = pd.read_excel(file_path, engine='openpyxl')
df.tail()

,Počty podnikatelů dle občanství podnikajících v České republice,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
161,Jemen,887,YE,YEM,10,1,0,2,0,3,1,0,0,1,3,1,1,0,23
162,Srbsko a Černá Hora,891,CS,SCG,259,54,16,15,10,24,11,5,1,2,21,8,15,7,448
163,Zambie,894,ZM,ZMB,1,2,0,0,0,0,0,0,0,0,0,0,0,0,3
164,Součet,SUM1,NaN,NaN,328403,257904,123176,108217,55388,131755,91623,106193,90211,85226,218492,106506,187649,108161,1998904
165,Z toho cizinci,SUM2,NaN,NaN,31712,9528,2852,5507,4934,7874,2598,2546,2160,1532,7097,2118,4124,1814,86396


In [6]:
df = df[3:]
df.head()

,Počty podnikatelů dle občanství podnikajících v České republice,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
3,Občanství,Kód,Kód zn.,Kód zn.,Hlavní město Praha,Středočeský kraj,Jihočeský kraj,Plzeňský kraj,Karlovarský kraj,Ústecký kraj,Liberecký kraj,Královéhradecký kraj,Pardubický kraj,Kraj Vysočina,Jihomoravský kraj,Olomoucký kraj,Moravskoslezský kraj,Zlínský kraj,Součet
4,Afghánistán,004,AF,AFG,16,7,0,0,1,6,2,0,0,1,13,4,2,0,52
5,Albánie,008,AL,ALB,13,8,4,1,1,3,1,2,1,1,2,1,1,4,43
6,Alžírsko,012,DZ,DZA,94,22,3,0,1,7,2,1,1,1,9,2,2,0,145
7,Angola,024,AO,AGO,9,2,2,0,0,3,2,1,1,0,6,3,2,2,33


In [7]:
# Automatizace převodu celé složky .XLSX na správné .CSV
# Otevře, převede na .csv, odebere 4 první řádky, 
# resetuje index, uloží jako .csv


processed = 0
all_dataframes = []
all_sums = []

# Roztřídit soubory v ascending order
folder_path = Path(download_path)
sorted_files = sorted(folder_path.iterdir(), key=lambda x: x.name)

# Prochází všechny soubory ve složce
for i, file in enumerate(sorted_files, start=0):  # Adding index i, start from 1
    if file.is_file() and not file.name.startswith('.'):  # Kontrola, zda jde o soubor
        
        # Definice nových názvů
        file_path = f"{download_path}/{file.name}"  # Proměnná filepath
        newname = file.name.replace(".xlsx", "")  # Odstranění .xlsx ze starého názvu
        csv_path = f"{export_path}/{newname}.csv"  # Nový název pro uložení
        summed_path = f"{export_path}/{newname}_{i}.csv"  # Using i as iteration number

        # Otevře excel v pandasu
        df = pd.read_excel(file_path, engine='openpyxl')  # Načíst excel jako pandas df

        # Stav
        processed += 1
        print(f"{processed} Soubor {csv_path} úspěšně převeden", end=" ")

        # Definice proměnných q a rok
        q = newname[-1]
        rok = newname[-6:-2]


        # Vytvoříme a upravíme df "sumcolumn" - sloupec pouze se součtem
        sumcolumn = df.iloc[:, [-1]]  #----> Uloží poslední sloupec pro účely dalšího bloku
        sumcolumn.insert(0, "Občanství", df.iloc[:, 0])  # Přidá sloupec "Občanství"
        sumcolumn = sumcolumn[3:].reset_index(drop=True) # Odstraní první 4 řádky
        sumcolumn.columns = sumcolumn.iloc[0]  # Nastaví první řádek jako názvy sloupců
        sumcolumn = sumcolumn[1:-2]  # Odstraní řádek použitý jako názvy sloupců + poslední řádek "Součet"
        sumcolumn = sumcolumn.set_index("Občanství")  # Nastaví sloupec "Občanství" jako index

        # Přidání suffixu do názvu sloupců
        for col in sumcolumn.columns:
            sumcolumn.rename(columns={col: f"{rok} Q{q}"}, inplace=True)


        # Zahodíme 6 zbytečných řádků, poslední sloupec + resetujeme index
        df = df[3:].reset_index(drop=True) # Odstraní první 4 řádky
        df = df.iloc[:,:-1]  # Odstraní poslední sloupec
        df = df[:-2]  # Odstraní poslední 2 řádky
        df.columns = df.iloc[0]  # Nastaví 5. řádek (který je teď první po odstranění) jako názvy sloupců
        df = df[1:]  # Odstraní řádek použitý jako názvy sloupců
        df = df.set_index("Občanství")  # Nastaví sloupec "Občanství" jako index
        df = df.sort_values(by=df.columns[0], ascending=True)  # Utřídí názvy řad abecedně

        # Přidání suffixu do názvu sloupců
        for i, col in enumerate(df.columns):
            if i >= 3:  # Začíná na 4. sloupci
                df.rename(columns={col: f"{col} {rok} Q{q}"}, inplace=True)


        # Přidat do listů
        all_dataframes.append(df)  # Přidat právě zpracovaný df do all_dataframes
        all_sums.append(sumcolumn)  # Přidat poslední sloupec do tohoto hovna

        # Uloží jako CSV
        sumcolumn.to_csv(summed_path, index=True)
        df.to_csv(csv_path, index=True)
        shape = df.shape
        print(f"+ uložen + přidán df do listu. Shape: {shape}", end="\n")


print(f"Ve složce je {processed} souborů.")

# Část s mergem v další buňce


1 Soubor ./Download_CSV/2017 Q1.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
2 Soubor ./Download_CSV/2017 Q2.csv úspěšně převeden + uložen + přidán df do listu. Shape: (161, 17)
3 Soubor ./Download_CSV/2017 Q3.csv úspěšně převeden + uložen + přidán df do listu. Shape: (162, 17)
4 Soubor ./Download_CSV/2017 Q4.csv úspěšně převeden + uložen + přidán df do listu. Shape: (161, 17)
5 Soubor ./Download_CSV/2018 Q1.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
6 Soubor ./Download_CSV/2018 Q2.csv úspěšně převeden + uložen + přidán df do listu. Shape: (161, 17)
7 Soubor ./Download_CSV/2018 Q3.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
8 Soubor ./Download_CSV/2018 Q4.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
9 Soubor ./Download_CSV/2019 Q1.csv úspěšně převeden + uložen + přidán df do listu. Shape: (160, 17)
10 Soubor ./Download_CSV/2019 Q2.csv úspěšně převeden + uložen + přidán df do listu. Shape:

In [9]:
len(all_sums)

31

In [187]:
'''
# Zjistíme, kde se děje problém s prázdnými poli v prvních 4 sloupcích
# prohledává už spojený soubor merged_df

nan_rows = merged_df.iloc[:, :4].isna().any(axis=1)
nan_indexes = merged_df[nan_rows].index.tolist()
print("Indexy s prázdným polem v jednom z prvních 4 sloupců:")
print(nan_indexes)
'''


Index s prázdným polem v jednom z prvních 4 sloupců:
['Velká Británie', 'Nizozemsko', 'Makedonie', 'Demokratická republika Kongo', 'Namibie', 'Curacao']


In [10]:
# Merge všech CSV do jednoho
# Vyplnit 0 na místo všech chybějících hodnot
# Ověřit, jestli nějaký řádek nemá všude 0, pokud ano -> smazat


# Loop through the list of DataFrames and merge them progressively
merged_df = all_dataframes[0]  # Start with the first DataFrame
merged_sums = all_sums[0]  # Start with the first DataFrame

# Merge progressively "merged_sums"
for df in all_sums[1:]:
    merged_sums = merged_sums.merge(df, how="outer", left_index=True, right_index=True)

# Merge progressively "merged_df"
for df in all_dataframes[1:]:
    # Merge only the columns that start from the 5th column onwards
    merged_df = merged_df.merge(df.iloc[:, 4:], how="outer", left_index=True, right_index=True)

# Doplnění 0 a převod typů na int
merged_df = merged_df.fillna(0)
merged_sums = merged_sums.fillna(0)

# Najít jestli nějaké země mají všude 0  "merged_df"
rows_all_zero = merged_df.iloc[:, 4:][merged_df.iloc[:, 4:].eq(0).all(axis=1)]  # Řádky s hodnotou 0 na všech pozicích
rows_to_delete = rows_all_zero.index.tolist()
merged_df.drop(rows_to_delete, inplace=True)  # Smazat řádky s hodnotou 0 všude

if rows_to_delete:
    print("merged_df - smazány řádky: ", end="")
    for i in rows_to_delete:
        print(i, end=", ")
else:
    print("merged_df - nenalezeny žádné řádky s 0")

# Najít jestli nějaké země mají všude 0  "merged_sums"
#rows_all_zero = merged_sums.iloc[:, 4:][merged_sums.iloc[:, 4:].eq(0).all(axis=1)]  # Řádky s hodnotou 0 na všech pozicích
#rows_to_delete = rows_all_zero.index.tolist()
#merged_sums.drop(rows_to_delete, inplace=True)  # Smazat řádky s hodnotou 0 všude

if rows_to_delete:
    print("merged_sums - smazány řádky: ", end="")
    for i in rows_to_delete:
        print(i, end=", ")
else:
    print("merged_sums - nenalezeny žádné řádky s 0")

# Utřídit "merged_sums" podle hodnot
merged_sums = merged_sums.sort_values(by=merged_sums.columns[0], ascending=False)  # Utřídí názvy řad abecedně

# Optional: Save the merged  "merged_df"
merged_df.to_csv(merged_csv_path, index=True)
print(f"\nVšechny soubory sloučeny do: {merged_csv_path}", end="\n\n\n")
# Optional: Save the merged  "merged_sums"
merged_sums.to_csv(merged_sums_path, index=True)
print(f"\nVšechny soubory sloučeny do: {merged_sums_path}", end="\n\n\n")


merged_df - smazány řádky: Americká Samoa, Americké Panenské ostrovy, Andorra, Anguilla, Antarktida, Antigua a Barbuda, Aruba, Bahamy, Bahrajn, Barbados, Belize, Bermudy, Bhútán, Bonaire, Svatý Eustach a Saba, Bouvetův ostrov, Britské indickooceánské území, Brunej, Cookovy ostrovy, Curaçao, Dominika, Džibutsko, Eritrea, Faerské ostrovy, Falklandy (Malvíny), Francouzská Polynésie, Francouzská jižní a antarktická území, Gabon, Gibraltar, Grenada, Grónsko, Guadeloupe, Guam, Guinea-Bissau, Guyana, Haiti, Heardův ostrov a MacDonaldovy ostrovy, Jersey, Jižní Georgie a Jižní Sandwichovy ostrovy, Kajmanské ostrovy, Katar, Kiribati, Kokosové (Keelingovy) ostrovy, Komory, Lesotho, Macao, Malawi, Man, Marshallovy ostrovy, Martinik, Mayotte, Menší odlehlé ostrovy USA, Mikronésie, Montserrat, Mosambik, Nauru, Niue, Nizozemské Antily, Norfolk, Nová Kaledonie, Omán, Palau, Papua Nová Guinea, Pitcairn, Portoriko, Rovníková Guinea, Réunion, Saint Pierre a Miquelon, Samoa, San Marino, Severní Mariany, S

/var/folders/dj/72cg9gkd69z_1134dyl3zsqw0000gn/T/ipykernel_99328/2741215437.py:20: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df = merged_df.fillna(0)
/var/folders/dj/72cg9gkd69z_1134dyl3zsqw0000gn/T/ipykernel_99328/2741215437.py:21: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_sums = merged_sums.fillna(0)


In [15]:
merged_sums
#merged_df

,017 Q1,017 Q2,017 Q3,017 Q4,018 Q1,018 Q2,018 Q3,018 Q4,019 Q1,019 Q2,...,022 Q2,022 Q3,022 Q4,023 Q1,023 Q2,023 Q3,023 Q4,024 Q1,024 Q2,024 Q4
Občanství,,,,,,,,,,,,,,,,,,,,,
Česko,1912508,1916065,1921817,1922699,1926393,1930027,1936350,1938653,1943284,1948734,...,2014474,2019838,2000040,1829874,1788680,1787711,1788261,1789125,1781538,1795537
Vietnam,22303,22182,22012,21773,21607,21500,21337,21172,21049,20904,...,20690,20721,20745,20730,20723,20786,20812,20890,20977,21145
Ukrajina,22136,22087,21926,21746,21730,21830,21762,21881,22065,22408,...,27283,29158,30978,30167,31955,32769,34497,33570,38144,43305
Slovensko,15722,16033,16304,16516,16809,17150,17473,17711,18119,18410,...,22324,22686,22983,23118,23375,23725,24058,24556,24923,25817
Německo,3104,3109,3105,3120,3131,3157,3163,3161,3129,3091,...,3137,3150,3192,3191,3213,3264,3298,3348,3367,3347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Monako,0,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Curaçao,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Mikronésie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
merged_df.shape

(166, 407)

In [17]:
merg = pd.read_csv(f"{export_path}/merged_dataset.csv")
merg.shape

(166, 408)

In [273]:
''''
JEDNORÁZOVÉ SPRAVENÍ SOUBORU 2016_2.csv

file_path = 'Down/2016_2.csv'  # Replace with your file path
df = pd.read_csv(file_path, sep=';')

# Save the file with commas as the delimiter
output_path = 'fixed_file.csv'  # Replace with your desired output path
df.to_csv(output_path, index=False, sep=',')
'''

In [ ]:
'''
------------------------------ THE END ---------------------------------
'''